# 태스크 1: Amazon Bedrock의 RetrieveAndGenerate API와 함께 완전관리형 RAG 애플리케이션 활용

이 태스크에서는 Amazon Bedrock의 RetrieveAndGenerate API와 함께 완전관리형 검색 증강 생성(RAG) 애플리케이션을 활용합니다.

다음 리소스는 실습 프로비저닝의 일부로 기존에 생성되었습니다.

- OpenSearch 서버리스 인덱스
- Amazon Bedrock의 지식 기반
- Amazon Bedrock의 지식 기반 내에 있는 데이터 소스

일반적인 데이터 수집 워크플로는 다음과 같습니다.

<!-- ![data_ingestion.png](images/data_ingestion.png) -->
<img src="images/data_ingestion.png" width=50% height=20% />

**이미지 설명: 위 다이어그램은 실습 환경의 일반적인 데이터 수집 워크플로를 보여줍니다.**

데이터 파이프라인은 일반적으로 여러 데이터 소스에 저장되는 문서를 지식 기반, 즉 Amazon OpenSearch Service Serverless(AOSS) 같은 벡터 데이터베이스로 수집하여 질문을 받았을 때 조회할 수 있도록 합니다.

- 문서는 다양한 데이터 소스(예: S3, Confluence, Sharepoint, Salesforce, 웹)에 연결하여 Amazon Bedrock의 지식 기반에 로드됩니다. 
- 그런 다음, 지식 기반은 선택된 전략에 따라 이를 더 작은 청크로 나누고 관련 벡터 저장소에 저장되는 임베딩을 생성합니다.

Bedrock 지식 기반에서 데이터를 사용할 수 있게 되면, Amazon Bedrock에서 제공하는 지식 기반 API를 사용하여 질문 답변 애플리케이션을 구축해 보겠습니다.

### 최종 실습 아키텍처:

<!-- ![retrieveAndGenerate.png](images/retrieveAndGenerate.png) -->
<img src="images/retrieveAndGenerate.png" width=50% height=20% />

*이미지 설명: 위 다이어그램에서는 RetrieveAndGenerate API를 사용하는 사용자 쿼리로 시작한 후, Amazon Bedrock에 있는 지식 기반의 문서를 사용하여 응답을 생성하는 데이터 흐름을 보여줍니다.*

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **주의:** **Run** 메뉴에서 **Run All Cells** 옵션을 사용하는 것보다 각 코드 셀을 개별적으로 실행하는 것이 좋습니다. 모든 셀을 한꺼번에 실행하면 커널 충돌 또는 재시작 같은 예기치 않은 동작이 발생할 수 있습니다. 셀을 하나씩 실행하면 실행 흐름을 더욱 잘 제어하고, 잠재적인 오류를 조기에 발견하며, 의도한 대로 코드가 실행되도록 할 수 있습니다.

## 태스크 1.1: 환경 설정

이 태스크에서는 boto3 클라이언트를 초기화하고 환경을 설정합니다. 노트북 전체에서 **RetrieveAndGenerate** API를 활용하여 지식 기반 기능을 테스트할 수 있습니다.

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i>**자세히 알아보기**: 자세한 내용은 **[RetrieveAndGenerate](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerate.html)** 을 참조하십시오.

1. 다음의 코드 셀을 실행하여 boto3 클라이언트를 초기화하고 환경을 설정합니다.

In [1]:
import json
import boto3
import pprint
from botocore.exceptions import ClientError
from botocore.client import Config

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "amazon.nova-lite-v1:0" # we will be using Amazon Nova lite throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

2. 다음의 코드 셀을 실행하여 Amazon Bedrock에 있는 기존 지식 기반의 ID를 확인합니다.

In [2]:
import botocore

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

Knowledge Base ID: V8BXZONLNU


## 태스크 1.2: RetrieveAndGenerate API를 사용하여 지식 기반 테스트

이 태스크에서는 **[retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html)** 함수를 사용하여 지식 기반을 테스트합니다.

처음에는 **RetrieveAndGenerate** API를 사용하여 지식 기반을 테스트합니다. 이 API를 사용하면 Amazon Bedrock은 지식 기반에서 필요한 참조를 검색한 후 Amazon Bedrock의 파운데이션 모델을 사용하여 최종 답변을 생성합니다.

3. 다음의 코드 셀 2개를 실행하여 지식 기반 테스트를 위한 쿼리를 시작합니다.

In [3]:
query = "Provide a summary of consolidated statements of cash flows of AnyCompany Financial for the fiscal years ended December 31, 2019?"

In [4]:
response = bedrock_agent_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":3
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

Answer: Based on the search results, here is a summary of the consolidated statements of cash flows of AnyCompany Financial for the fiscal year ended December 31, 2019 :

- Cash flows from operating activities: AnyCompany Financial had net cash provided by operating activities of $710 million in 2019. The increase in net cash provided by operating activities was primarily due to an increase in net income and favorable changes in operating assets and liabilities.

- Cash flows from investing activities: Cash flows from investing activities in 2019 resulted in net cash used of $240 million. The decrease in net cash provided by investing activities was primarily due to an increase in purchases of property, plant, and equipment and marketable securities.

- Cash flows from financing activities: In 2019, AnyCompany Financial had net cash provided by financing activities of $350 million. The increase in net cash provided by financing activities was primarily due to an increase in proceeds fr

## 태스크 1.3: RetrieveAndGenerate API 파악

이 태스크에서는 RetrieveAndGenerate API에 대해 파악합니다.

**RetrieveAndGenerate** API는 다음과 같은 2가지 중요한 파라미터를 사용합니다.

- **numberOfResults**: 지정된 함수의 **numberOfResults** 파라미터는 검색 결과의 수를 결정합니다. 이러한 검색 결과는 지식 기반에서 검색된 후 답변 생성을 위해 모델에 제공된 프롬프트에 포함됩니다. 특히, 주어진 쿼리와 가장 밀접하게 일치하는 문서 또는 검색 결과의 최상위 **max_results** 수를 가져옵니다.

- **textPromptTemplate**: **textPromptTemplate** 파라미터는 모델에 제공된 프롬프트의 템플릿 역할을 하는 문자열입니다. 이 경우에는 **default_prompt**가 템플릿으로 사용됩니다. 이 템플릿에는 자리 표시자(`$search_results$` 및 `$output_format_instructions$`)가 포함되어 있으며 이는 모델에 전달되기 전에 각각 실제 검색 결과 및 출력 형식 지침으로 대체됩니다.

4. 다음의 코드 셀 2개를 실행하여 지식 기반의 기본 프롬프트를 명시합니다.

In [5]:
# Stating the default knowledge base prompt
default_prompt = """
You are a question answering agent. I will provide you with a set of search results.
The user will provide you with a question. Your job is to answer the user's question using only information from the search results. 
If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question. 
Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
                            
Here are the search results in numbered order:
$search_results$


$output_format_instructions$

"""

In [6]:
def retrieve_and_generate(query, kb_id, model_arn, max_results, prompt_template = default_prompt):
    response = bedrock_agent_client.retrieve_and_generate(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn, 
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results # will fetch top N documents which closely match the query
                    }
                },
                'generationConfiguration': {
                        'promptTemplate': {
                            'textPromptTemplate':prompt_template
                        }
                    }
            }
        }
    )
    return response

## 태스크 1.4: 최대 결과 수 기능 활용

이 태스크에서는 최대 결과 수 기능을 활용합니다. 일부 사용 사례의 경우 FM에서 도출된 응답에 관련 답변을 제공할 만한 컨텍스트가 부족하거나, 이러한 응답으로는 요청한 정보를 찾지 못할 수 있습니다. 이와 같은 문제는 최대 검색 결과 수를 수정하여 해결할 수 있습니다.

여기서는 하나의 결과를 사용하여 다음 쿼리를 실행합니다.

```Provide a list of risks for AnyCompany financial in bulleted points.```

5. 다음의 코드 셀 2개를 실행합니다.

In [7]:
def print_generation_results(response, print_context = True):
    generated_text = response['output']['text']
    print('Generated FM response:\n')
    print(generated_text)
    
    if print_context is True:
        ## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
        citations = response["citations"]
        contexts = []
        for citation in citations:
            retrievedReferences = citation["retrievedReferences"]
            for reference in retrievedReferences:
                contexts.append(reference["content"]["text"])
    
        print('\n\n\nRetrieved Context:\n')
        pprint.pp(contexts)


In [8]:
query = """Provide a list of risks for AnyCompany financial in numbered list without description."""

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 1)

print_generation_results(results)

Generated FM response:

Here is a list of risks for AnyCompany Financial based on the search results:

1. Commodity Prices 2. Foreign Exchange Rates 3. Equity Prices These risks could potentially have a material adverse effect on AnyCompany Financial's revenue, profitability, cost of goods sold, and investment portfolio.



Retrieved Context:

['However, the company still faces significant risks if any of its key '
 "suppliers were to fail.      ### Market Risks      AnyCompany Financial's "
 'financial performance is subject to various market risks, including changes '
 'in commodity prices, foreign exchange rates, and equity prices. For example, '
 'a significant decline in the price of oil could have a material adverse '
 "effect on AnyCompany Financial's revenue and profitability. Similarly, "
 "changes in foreign exchange rates could impact the company's cost of goods "
 'sold and profitability.      ### Table: Market Risks      | Risk Factor | '
 'Description | Potential Impact |

6. 검색된 결과 수에 대한 수정된 값을 사용하여 다음 코드 셀을 실행합니다.

In [9]:
#Using higher number of max results

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3)

print_generation_results(results)

Generated FM response:

Here is a list of risks for AnyCompany Financial in a numbered list format:

1. Market Risks
2. Operational Risk
3. Regulatory Risk
4. Strategic Risk Note: The descriptions of these risks are not provided in the search results.



Retrieved Context:

['Our liquidity management policies aim to minimize this risk, but we cannot '
 'eliminate it entirely. For instance, PersonC, our Treasurer, mentioned that '
 'during the COVID-19 pandemic, the market for commercial mortgage- backed '
 'securities (CMBS) became illiquid, making it challenging to sell these '
 'securities at fair market values.      ### Operational Risk      AnyCompany '
 'Financial is exposed to operational risk due to the possibility of losses '
 'resulting from inadequate or failed internal processes, people, and systems, '
 'or from external events. Our risk management policies and procedures aim to '
 'minimize this risk, but we cannot eliminate it entirely. For example, we '
 'have experienced

출력에서 볼 수 있듯이 검색된 결과 수를 **3**으로 수정하면 더 많은 결과를 얻을 수 있어 보다 포괄적인 응답을 얻을 수 있습니다.

## 태스크 1.5: 사용자 지정 프롬프트 기능 사용

이 태스크에서는 사용 사례에 따라 고유한 프롬프트로 기본 프롬프트를 사용자 지정합니다. 이 기능은 특정 출력 형식, 언어 및 기타 컨텍스트가 필요한 FM에 더 많은 컨텍스트를 추가하는 데 도움이 됩니다. 

Langchain의 *PromptTemplate* 클래스를 사용하여 *custom_prompt* 템플릿을 파라미터화합니다. *output_format * 변수를 사용하면 실행 시간에 출력 형식을 수정할 수 있습니다.

7. 다음 코드 셀을 실행하여 템플릿 사용자 지정을 지원하는 도우미 기능을 생성합니다.

In [10]:
from langchain.prompts import PromptTemplate
def format_prompt(input_prompt:str, output_format:str):
    formatted_prompt: List[str] = []
    prompt = PromptTemplate.from_template(input_prompt)     
    formatted_prompt.extend(prompt.format(output_format=output_format))
    return "".join(formatted_prompt)


### 태스크 1.5.1: 동일한 쿼리 예제를 사용하고 FM을 기본 설정하여 다른 언어로 출력합니다

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 기본 프롬프트에서 ```$output_format_instructions$```를 제거하면 생성된 응답에서 인용이 삭제됩니다. 하지만 `output_format`은 런타임 출력 사용자 정의에는 도움이 됩니다.

8. 다음의 코드 셀 3개를 실행하여 프랑스어로 출력을 제공하는 예제를 테스트합니다.

In [11]:
## Example 1
custom_prompt = """
You are a question answering agent. I will provide you with a set of search results. 
The user will provide you with a question. Your job is to answer the user's question using only information from the search results.
If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question.
Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
                            
Here are the search results in numbered order:
$search_results$


{output_format}
"""

In [15]:
print(format_prompt(custom_prompt, "한국어로 답을 해줘."))


You are a question answering agent. I will provide you with a set of search results. 
The user will provide you with a question. Your job is to answer the user's question using only information from the search results.
If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question.
Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
                            
Here are the search results in numbered order:
$search_results$


한국어로 답을 해줘.



In [17]:
results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3, prompt_template = format_prompt(custom_prompt, "한국어로 답을 해줘."))

print_generation_results(results, print_context = False)

Generated FM response:

1. Market Risks
2. Operational Risk
3. Regulatory Risk
4. Strategic Risk



### 태스크 1.5.2: 동일한 쿼리 예제를 사용하고 JSON 형식으로 출력하도록 FM의 기본값을 설정합니다.

9. 다음의 코드 셀을 실행하여 JSON 형식의 출력을 제공하는 예제를 테스트합니다.

In [18]:
results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3, prompt_template = format_prompt(custom_prompt, "Please provide your response using a JSON format."))

print_generation_results(results,print_context = False)

Generated FM response:

{"risks":["Market Risks","Operational Risk","Regulatory Risk","Strategic Risk"]}


<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **태스크 완료:** 이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 태스크 2를 계속 진행합니다.
